### Summary of notebook

This notebook is made up of the following sections
1. Data Ingest
2. Data Wrangling
3. Data Exploration 
4. Modelling 


In [ ]:
# Load relevant libaries 
import pandas            as pd
import pandasql          as psql
import requests
from bs4                 import BeautifulSoup
import io
import matplotlib.pyplot as plt
import missingno         as msno

## Data Ingest 

Ingesting this data presents a unique challenge because it is dynamically uploaded via JavaScript, making standard web scraping methods ineffective. Consequently, three specialised functions have been wrote to handle the ingestion. 

Note: The scale of the data is 31 million rows and as such loading times can be extensive. Functions have docstrings; please consult them for additional information.

In [ ]:
def extract_csv_urls(url):
    """
    Function to extract CSV URLs from a webpage.

    Args:
    url (str): The URL of the webpage to scrape.

    Returns:
    list: A list of URLs pointing to CSV files found on the webpage.

    """
    csv_urls = []
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    for c in soup.select('contents key'):
        if c.text.startswith('usage-stats') and c.text.endswith('.csv'):
            if '2021' in c.text or '2022' in c.text or '2023' in c.text:
                csv_urls.append('https://cycling.data.tfl.gov.uk/' + c.text)
    return csv_urls

def read_csv_as_dataframe(csv_url):
    """
    Function to read a CSV file from a URL and return it as a DataFrame.

    Args:
    csv_url (str): The URL of the CSV file to read.

    Returns:
    pandas.DataFrame or None: A DataFrame containing the CSV data if successful, otherwise None.

    """
    response = requests.get(csv_url)
    # Check if the request was successful
    if response.status_code == 200:
        # Read CSV content into a DataFrame
        df = pd.read_csv(io.BytesIO(response.content))
        return df
    else:
        print("Failed to fetch CSV from URL:", csv_url)
        return None

def combine_csvs(csv_urls):
    """
    Function to combine multiple CSVs into a single DataFrame.

    Args:
    csv_urls (list): A list of URLs pointing to CSV files.

    Returns:
    pandas.DataFrame: A DataFrame containing the combined data from all CSV files.

    """
    dataframes = []
    for url in csv_urls:
        data = read_csv_as_dataframe(url)
        if data is not None:
            dataframes.append(data)
    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df


In [ ]:
# Ingest Data 
webpage_url = 'https://s3-eu-west-1.amazonaws.com/cycling.data.tfl.gov.uk/?list-type=2&max-keys=1500'
csv_urls = extract_csv_urls(webpage_url)

# If Statement to highlight if the functions have been unsuccessful
if csv_urls:
    data = combine_csvs(csv_urls)
    print("Combined DataFrame from all CSV files:")
else:
    print("No CSV URLs found.")

## Data Wrangling

This section includes 
1. Viewing the data 
2. Visualise Nulls 
3. Identify and fix Null introduction 
4. Cast data types correctly 
5. Check all dates are within 2021-2023 

In [13]:
# Quick view shows data appears to have nulls
data.head(10)

# Count nulls by columns. No clear pattern nulls across multiple columns. 
for column in data.columns:
    null_count = data[column].isnull().sum()
    print(f"Column {column}: {null_count} null values")
    
# Find Min/Max Start Date vs Start_Date. 

# Results show that nulls are the result of changing column names on date 2022-09-19. 
# Columns can be mapped together to create a full data set.

# Some column names share similar naming conventions. A postfix has been added to distinguish them.
postfix = '_2019'

# Identify columns 10-20 by their positional indices as the columns to be postfixed
cols_to_postfix = data.columns[9:20]

# Apply the postfix to the selected column names
new_columns = [col + postfix if col in cols_to_postfix else col for col in subset_data.columns]

# Assign the new column names back to the DataFrame
data.columns = new_columns

# SQL interface used to create a unified data frame without nulls. 
# TODO: Need to cast data types. 
# TODO: Check the station ids is this ok.

query = """
SELECT  
    CASE 
        WHEN "Rental Id" IS NULL THEN CAST("Number_2019" AS VARCHAR)
        ELSE CAST("Rental Id" AS VARCHAR)
    END AS "ID",
    CASE 
        WHEN "Duration" IS NULL 
        THEN "Total duration (ms)_2019"/1000 
        ELSE "Duration" 
    END AS Ride_Duration_Seconds,
    CASE WHEN "Bike ID" IS NULL 
        THEN "Bike number_2019" 
        ELSE "Bike ID" 
    END AS Bike_ID, 
    CASE 
        WHEN "Start Date" IS NULL 
        THEN "Start date_2019" 
        ELSE "Start Date" 
    END AS Start_Date, 
    CASE 
        WHEN "End Date" IS NULL 
        THEN "End date_2019" 
        ELSE "End Date"
    END AS End_Date,
    CASE 
        WHEN "StartStation Name" IS NULL 
        THEN "Start station_2019" 
        ELSE "StartStation Name" 
    END AS Start_Station_Name, 
    CASE 
        WHEN "EndStation Name" IS NULL 
        THEN "End station_2019" 
        ELSE "EndStation Name"
    END AS End_Station_Name,
    CASE 
        WHEN "StartStation Id" IS NULL 
        THEN "Start station number_2019" 
        ELSE "StartStation Id" 
    END AS Start_Station_ID, 
    CASE 
        WHEN "EndStation Id" IS NULL 
        THEN "End station number_2019" 
        ELSE "EndStation Id"
    END AS End_Station_ID
FROM subset_data
"""

# Query run
complete_data = psql.sqldf(query, locals())

# Rerun null count 
# Count nulls by columns. No clear pattern nulls across multiple columns. 
for column in complete_data.columns:
    null_count = complete_data[column].isnull().sum()
    print(f"Column {column}: {null_count} null values")

## Wip Wrangling 

In [ ]:
subset_data = pd.read_csv('subset_data.csv')
subset_data.head(10)

In [ ]:
# Postfix for column names
postfix = '_2019'

# Identify columns 10-12 by their positional indices (9, 10, 11 in 0-based indexing)
cols_to_postfix = subset_data.columns[9:20]

# Apply the postfix to the selected column names
new_columns = [col + postfix if col in cols_to_postfix else col for col in subset_data.columns]

# Assign the new column names back to the DataFrame
subset_data.columns = new_columns

In [ ]:
subset_data

In [ ]:
# Sorts out column descrepancies. Need to cast data types. Check the station ids is this ok.

query = """
SELECT  
    CASE 
        WHEN "Rental Id" IS NULL THEN CAST("Number_2019" AS VARCHAR)
        ELSE CAST("Rental Id" AS VARCHAR)
    END AS "ID",
    CASE 
        WHEN "Duration" IS NULL 
        THEN "Total duration (ms)_2019"/1000 
        ELSE "Duration" 
    END AS Ride_Duration_Seconds,
    CASE WHEN "Bike ID" IS NULL 
        THEN "Bike number_2019" 
        ELSE "Bike ID" 
    END AS Bike_ID, 
    CASE 
        WHEN "Start Date" IS NULL 
        THEN "Start date_2019" 
        ELSE "Start Date" 
    END AS Start_Date, 
    CASE 
        WHEN "End Date" IS NULL 
        THEN "End date_2019" 
        ELSE "End Date"
    END AS End_Date,
    CASE 
        WHEN "StartStation Name" IS NULL 
        THEN "Start station_2019" 
        ELSE "StartStation Name" 
    END AS Start_Station_Name, 
    CASE 
        WHEN "EndStation Name" IS NULL 
        THEN "End station_2019" 
        ELSE "EndStation Name"
    END AS End_Station_Name,
    CASE 
        WHEN "StartStation Id" IS NULL 
        THEN "Start station number_2019" 
        ELSE "StartStation Id" 
    END AS Start_Station_ID, 
    CASE 
        WHEN "EndStation Id" IS NULL 
        THEN "End station number_2019" 
        ELSE "EndStation Id"
    END AS End_Station_ID
FROM subset_data
"""

# Running the query
result = psql.sqldf(query, locals())

print(result)

In [ ]:
# Postfix for column names
postfix = '_2019'

# Identify columns 10-12 by their positional indices (9, 10, 11 in 0-based indexing)
cols_to_postfix = combined_df.columns[9:20]

# Apply the postfix to the selected column names
new_columns = [col + postfix if col in cols_to_postfix else col for col in combined_df.columns]

# Assign the new column names back to the DataFrame
combined_df.columns = new_columns

In [ ]:
# Sorts out column descrepancies. Need to cast data types. Check the station ids is this ok.

query = """
SELECT  
    CASE 
        WHEN "Rental Id" IS NULL THEN CAST("Number_2019" AS VARCHAR)
        ELSE CAST("Rental Id" AS VARCHAR)
    END AS "ID",
    CASE 
        WHEN "Duration" IS NULL 
        THEN "Total duration (ms)_2019"/1000 
        ELSE "Duration" 
    END AS Ride_Duration_Seconds,
    CASE WHEN "Bike ID" IS NULL 
        THEN "Bike number_2019" 
        ELSE "Bike ID" 
    END AS Bike_ID, 
    CASE 
        WHEN "Start Date" IS NULL 
        THEN "Start date_2019" 
        ELSE "Start Date" 
    END AS Start_Date, 
    CASE 
        WHEN "End Date" IS NULL 
        THEN "End date_2019" 
        ELSE "End Date"
    END AS End_Date,
    CASE 
        WHEN "StartStation Name" IS NULL 
        THEN "Start station_2019" 
        ELSE "StartStation Name" 
    END AS Start_Station_Name, 
    CASE 
        WHEN "EndStation Name" IS NULL 
        THEN "End station_2019" 
        ELSE "EndStation Name"
    END AS End_Station_Name,
    CASE 
        WHEN "StartStation Id" IS NULL 
        THEN "Start station number_2019" 
        ELSE "StartStation Id" 
    END AS Start_Station_ID, 
    CASE 
        WHEN "EndStation Id" IS NULL 
        THEN "End station number_2019" 
        ELSE "EndStation Id"
    END AS End_Station_ID
FROM combined_df
"""

# Running the query
result = psql.sqldf(query, locals())

print(result)

In [ ]:
result

In [ ]:
# Null Visualisation 

# msno.matrix(subset_data)
msno.matrix(result)

# Data Exploration 

We now have a fully formed data frame the requirement of the client from this eda is as follows 

Highlighted 3 main areas 
1. What kinds of usage they might expect to see 
2. What profile of customers they might have 
3. Operational concerns such as reliability and supply chain management.


## Usage 

In [ ]:
# SQL query to extract avg ride duration and total rides per year
query = """
SELECT  
    CASE 
        WHEN "End_Date" LIKE '%2021%' THEN '2021'
        WHEN "End_Date" LIKE '%2022%' THEN '2022'
        WHEN "End_Date" LIKE '%2023%' THEN '2023'
        ELSE 'Other'
    END AS Year,
    AVG(Ride_Duration_Seconds) AS Average_Ride_Duration,
    COUNT(*) as Total_Rides
FROM result
WHERE Year NOT LIKE 'Other'
GROUP BY 
    CASE 
        WHEN "End_Date" LIKE '%2021%' THEN '2021'
        WHEN "End_Date" LIKE '%2022%' THEN '2022'
        WHEN "End_Date" LIKE '%2023%' THEN '2023'
        ELSE 'Other'
    END
"""

# Run query and save to df
usage_date = psql.sqldf(query, locals())

# Calculate the YoY growth for both key metrics
usage_date['YoY_Growth_Duration'] = usage_date['Average_Ride_Duration'].pct_change() * 100
usage_date['YoY_Growth_Rides'] = usage_date['Total_Rides'].pct_change() * 100

# Add 'type' column. Needed for plotting actuals vs predicted.
usage_date['type'] = 'actual'

# Calculate average YoY growth over all years.
average_yoy_growth_duration = usage_date['YoY_Growth_Duration'].mean()
average_yoy_growth_rides = usage_date['YoY_Growth_Rides'].mean()

# Predict Average_Ride_Duration for 2024 using 2023 metrics * average YoY growth metric
predicted_2024_duration = usage_date.loc[usage_date['Year'] == '2023', 'Average_Ride_Duration'].values[0] * (1 + average_yoy_growth_duration / 100)
predicted_2024_rides = usage_date.loc[usage_date['Year'] == '2023', 'Total_Rides'].values[0] * (1 + average_yoy_growth_rides / 100)

# Create the 2024 Row using previous calculations
new_row = {
    'Year': '2024',
    'Average_Ride_Duration': predicted_2024_duration,
    'Total_Rides': predicted_2024_rides,
    'type': 'predicted'
}

# Append 2024 to existing dataframe and add avg yoy growth metrics.
usage_data_metrics = pd.concat([usage_date, pd.DataFrame([new_row])], ignore_index=True)
usage_data_metrics['avg_yearly_yoy_growth_duration'] = average_yoy_growth_duration
usage_data_metrics['average_yoy_growth_rides'] = average_yoy_growth_rides
usage_data_metrics.head(4)


### Usage Visualisation

In [ ]:
# Creating the figure and axes objects
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plotting Total Rides on the first y-axis
ax1.bar(usage_data_metrics['Year'], usage_data_metrics['Total_Rides'], color='#D85604', alpha=0.7, label='Total Rides')
ax1.set_xlabel('Year')
ax1.set_ylabel('Total Rides')
ax1.tick_params('y')

# Creating a second y-axis for Average Ride Duration
ax2 = ax1.twinx()
ax2.plot(usage_data_metrics['Year'], usage_data_metrics['Average_Ride_Duration'], color='black', marker='o', label='Average Ride Duration')
ax2.set_ylabel('Average Ride Duration', color='black')
ax2.tick_params('y', colors='black')

# Adding title and legend
fig.suptitle('Ride Usage Statistics Over Years')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Display the plot
plt.tight_layout()
plt.show()

### Customer Profile

Time Profile 
Destination Profile 

In [ ]:
# SQL query to extract count of rides at hourly aggregate grouped by whether weekend 

query = """
WITH dates AS (
    SELECT 
        SUBSTRING(Start_Date, 1, 10) AS date,
        SUBSTR(Start_Date, 12, 2) AS hour,
        *
    FROM 
        result
) 

SELECT
    *,
    CASE 
        WHEN INSTR(date, '-') > 0 THEN DATE(date) -- Format: yyyy-mm-dd
        WHEN INSTR(date, '/') > 0 AND LENGTH(date) = 10 THEN DATE(SUBSTR(date, 7, 4) || '-' || SUBSTR(date, 4, 2) || '-' || SUBSTR(date, 1, 2)) -- Format: mm/dd/yyyy
        WHEN INSTR(date, '/') > 0 AND LENGTH(date) = 8 THEN DATE(SUBSTR(date, 5, 4) || '-' || SUBSTR(date, 3, 2) || '-' || SUBSTR(date, 1, 2)) -- Format: dd/mm/yyyy
        ELSE NULL 
    END AS normalised_date
FROM dates 
"""

# Run query and save to df
time_data  = psql.sqldf(query, locals())

In [ ]:
# Weekend Flag 
time_data_cleaned = pd.DataFrame(time_data)
time_data_cleaned['normalised_date'] = pd.to_datetime(time_data['normalised_date'])
time_data_cleaned['is_weekend'] = time_data_cleaned['normalised_date'].dt.dayofweek // 5

In [ ]:
time_data_metrics = time_data_cleaned.groupby(['hour', 'is_weekend']).size().reset_index(name='count')
time_data_metrics['hour'] = pd.to_numeric(time_data_metrics['hour'])

In [ ]:
# Function to normalize the count within each group
def min_max_normalize(group):
    return (group - group.min()) / (group.max() - group.min())

# Apply the normalization within each group
time_data_metrics['count_normalised'] = time_data_metrics.groupby('is_weekend')['count'].transform(min_max_normalize)


### Customer Profile Visualisation

In [ ]:
# Define colors
color1 = '#D85604'  # Weekend color
color2 = '#000000'  # Weekday color

# Filter the data
weekend_data = time_data_metrics[time_data_metrics['is_weekend'] == True]
weekday_data = time_data_metrics[time_data_metrics['is_weekend'] == False]

# Plotting
fig, ax = plt.subplots(figsize=(12, 6))

# Plot weekend line
line_weekend, = ax.plot(weekend_data['hour'], weekend_data['count_normalised'], color=color1, label='Weekend')

# Plot weekday line
line_weekday, = ax.plot(weekday_data['hour'], weekday_data['count_normalised'], color=color2, label='Weekday')

# Adding labels and title
ax.set_xlabel('Hour')
ax.set_ylabel('Percentage of total rides by hour')
ax.set_title('Distribution of rides per hour categorised by weekend vs. weekday')

# Set x-ticks to show all hours
ax.set_xticks(range(24))
ax.set_xticklabels(range(24))

# Adding legend
ax.legend()

# Show plot
plt.show()



### Reliability

In [ ]:
# SQL query to extract count of rides at hourly aggregate grouped by whether weekend 

reliability_query = """
WITH dates AS (
    SELECT 
        *,
        SUBSTR(Start_Date, 1, 10) AS date
    FROM 
        result
) 
SELECT
    Bike_Id AS bike_id,
    MIN(date) AS first_use, 
    MAX(date) AS last_use,
    julianday(MAX(date)) - julianday(MIN(date)) AS date_difference
FROM dates
GROUP BY ID
"""

# Run query and save to df
reliability_data  = psql.sqldf(reliability_query, locals())

In [ ]:
reliability_data

In [ ]:


# Customer Profile 
# - Geographical e.g where people are coming from where they are going  
#       - Most common start point/end point 
# - Habitual Types of people 
#        - Most popular ride time by hour etc 
# DS Use Case: Segment customers based on cycling behaviour this could be used to inform marketing dynamic pricing etc. 

# Operational Concerns 
# Reliability             - When does bike ID stop appearing in the data e.g life of bike 
#                         - Avg Life span of a bike 
#                         - Location of bikes dying 
# DS Use Case: 

# Supply chain management - Hot spots of rides     

## Data Use Cases 

Idea 1 - Supply Chain Management: 
- Use heat maps to visualize high-demand areas and times.
- Analyze peak usage periods to ensure adequate bike availability.
- Predictive analytics to forecast demand and optimize bike placement.

Idea 2 - Habitual Types of Riders
- Segment riders into categories (e.g., early morning riders, weekend riders, commuters) based on trip start times.
- Use clustering algorithms to identify patterns in riding habits.

Idea 3 - Reliability
- Track the first and last appearance of each bike ID in the data.
- Calculate the operational lifespan of bikes.
- Identify patterns or reasons for bike retirements (e.g., frequent repairs, high usage).